# DES Algorithm Implementation
In this notebook, we will walk through the steps of implementing the Data Encryption Standard (DES) algorithm.

## Authors
[Abtin Zandi](https://github.com/Abtinz), [Mohammad Sadegh Mohammadi](https://github.com/sadegh-msm) 

## Organization 
[AUT-basics-of-security-fall-2024](https://github.com/AUT-basics-of-security-fall-2024) 

## Step 1: Initial Permutation (IP)
The first step in DES is to permute the 64-bit input block using a predefined table.

In [49]:
IP_TABLE = [58, 50, 42, 34, 26, 18, 10, 2, 60, 52, 44, 36, 28, 20, 12, 4,
            62, 54, 46, 38, 30, 22, 14, 6, 64, 56, 48, 40, 32, 24, 16, 8,
            57, 49, 41, 33, 25, 17, 9, 1, 59, 51, 43, 35, 27, 19, 11, 3,
            61, 53, 45, 37, 29, 21, 13, 5, 63, 55, 47, 39, 31, 23, 15, 7]

#this function will permute the initial inputted block
def permute(block, table):
    return ''.join([block[i - 1] for i in table])

input_block = '0000000100100011010001010110011110001001101010111100110111101111'
initial_permutation = permute(input_block, IP_TABLE)
print("Initial Permutation: ", initial_permutation)

Initial Permutation:  1100110000000000110011001111111111110000101010101111000010101010


Initial Permutation:  1100110000000000110011001111111111110000101010101111000010101010

## Step 2: Key Generation
We now generate the 56-bit key by ignoring every 8th bit of the original 64-bit key.

In [50]:
def generate_56bit_key(key):
    # Ignore every 8th bit (0-indexed positions 7, 15, 23, 31, 39, 47, 55)
    key_56bit = ''.join([key[i] for i in range(len(key)) if (i + 1) % 8 != 0])
    return key_56bit

key = '0001001100110100010101110111100110011011101111001101111111110001'
key_56bit = generate_56bit_key(key)
print("56-bit Key: ", key_56bit)

56-bit Key:  00010010011010010101101111001001101101111011011111111000


56-bit Key:  00010010011010010101101111001001101101111011011111111000

## Step 3: Key Splitting and Shifting
We split the 56-bit key into two 28-bit halves, and shift them according to the round number.

In [51]:
SHIFT_TABLE = [1, 1, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 1]

def shift_left(k, shifts):
    n = shifts % len(k)
    k = k[n:] + k[:n]
    return k

# here we should split the generated key into two halves
left_half, right_half = key_56bit[:28],key_56bit[28:]

print("left half: ", left_half)
print("right half: ", right_half)

left_half_shifted = shift_left(left_half,1)
right_half_shifted = shift_left(right_half,1)

print("Left Half Shifted: ", left_half_shifted)
print("Right Half Shifted: ", right_half_shifted)

left half:  0001001001101001010110111100
right half:  1001101101111011011111111000
Left Half Shifted:  0010010011010010101101111000
Right Half Shifted:  0011011011110110111111110001


left half:  0001001001101001010110111100

right half:  1001101101111011011111111000

Left Half Shifted:  0010010011010010101101111000

Right Half Shifted:  0011011011110110111111110001

## Step 4: Key Compression
We compress the shifted key halves into a 48-bit subkey.

In [52]:
COMPRESSION_TABLE = [14, 17, 11, 24, 1, 5, 3, 28, 15, 6, 21, 10, 23, 19, 12, 4,
                    26, 8, 16, 7, 27, 20, 13, 2, 41, 52, 31, 37, 47, 55, 30, 40,
                    51, 45, 33, 48, 44, 49, 39, 56, 34, 53, 46, 42, 50, 36, 29, 32]

# combine the left and right halves
combined_key = left_half_shifted + right_half_shifted

def compress_key(combined_key, table):
    return ''.join([combined_key[i - 1] for i in table])
# compressing the key to 48 bits using permutation function
subkey = compress_key(combined_key, COMPRESSION_TABLE)
print("Subkey: ", subkey)

Subkey:  010100101101111000000100011110011101011110111001


Subkey:  010100101101111000000100011110011101011110111001

## Rounds of Encryption
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

## Step 5: splitting the block
first section of encryption is splitting the block into two halves

In [53]:
#this function will split the initial_permutation into two separated blocks into two 32-bit block
def split_block(block):
    mid = len(block) // 2
    return block[:mid], block[mid:]

left_half, right_half = split_block(initial_permutation)
print("Left half of block: ", left_half)
print("Right half of block: ", right_half)

Left half of block:  11001100000000001100110011111111
Right half of block:  11110000101010101111000010101010


Left half of block:  11001100000000001100110011111111

Right half of block:  11110000101010101111000010101010

# Step 6:  
second part of encryption iterations is expanding the right half to 48 bits

In [54]:
EXPANSION_TABLE = [32, 1, 2, 3, 4, 5, 4, 5, 6, 7, 8, 9, 8, 9, 10, 11, 12, 13, 12, 13, 14, 15, 16, 17,
                   16, 17, 18, 19, 20, 21, 20, 21, 22, 23, 24, 25, 24, 25, 26, 27, 28, 29, 28, 29, 30, 31, 32, 1]

#this function will expand the right half of processing block to 48 bits with EXPANSION_TABLE
def expand_right_half(block, table):
    return ''.join([block[i - 1] for i in table])

expanded_right = expand_right_half(right_half, EXPANSION_TABLE)
print("expanded right half: ", expanded_right)

expanded right half:  011110100001010101010101011110100001010101010101


expanded right half:  011110100001010101010101011110100001010101010101

In [55]:
#after expanding and permuting the right half of given block, we should conduct xor process with round key
def xor_function(block, key):
    return ''.join(['0' if bit1 == bit2 else '1' for bit1, bit2 in zip(block, key)])

xor_val = xor_function(expanded_right, subkey)
print("expanded right half: ", xor_val)

expanded right half:  001010001100101101010001000000111100001011101100


expanded_xor_val right half:  001010001100101101010001000000111100001011101100

## Step 7: Processing through S-boxes

In [56]:
# substitution box tables for DES encryption process
S_BOX = [
    # first S-box
    [
        [14, 4, 13, 1, 2, 15, 11, 8, 3, 10, 6, 12, 5, 9, 0, 7],
        [0, 15, 7, 4, 14, 2, 13, 1, 10, 6, 12, 11, 9, 5, 3, 8],
        [4, 1, 14, 8, 13, 6, 2, 11, 15, 12, 9, 7, 3, 10, 5, 0],
        [15, 12, 8, 2, 4, 9, 1, 7, 5, 11, 3, 14, 10, 0, 6, 13]
    ],
    # second S-box
    [
        [15, 1, 8, 14, 6, 11, 3, 4, 9, 7, 2, 13, 12, 0, 5, 10],
        [3, 13, 4, 7, 15, 2, 8, 14, 12, 0, 1, 10, 6, 9, 11, 5],
        [0, 14, 7, 11, 10, 4, 13, 1, 5, 8, 12, 6, 9, 3, 2, 15],
        [13, 8, 10, 1, 3, 15, 4, 2, 11, 6, 7, 12, 0, 5, 14, 9]
    ],
    # third S-box
    [
        [10, 0, 9, 14, 6, 3, 15, 5, 1, 13, 12, 7, 11, 4, 2, 8],
        [13, 7, 0, 9, 3, 4, 6, 10, 2, 8, 5, 14, 12, 11, 15, 1],
        [13, 6, 4, 9, 8, 15, 3, 0, 11, 1, 2, 12, 5, 10, 14, 7],
        [1, 10, 13, 0, 6, 9, 8, 7, 4, 15, 14, 3, 11, 5, 2, 12]
    ],
    # fourth S-box
    [
        [7, 13, 14, 3, 0, 6, 9, 10, 1, 2, 8, 5, 11, 12, 4, 15],
        [13, 8, 11, 5, 6, 15, 0, 3, 4, 7, 2, 12, 1, 10, 14, 9],
        [10, 6, 9, 0, 12, 11, 7, 13, 15, 1, 3, 14, 5, 2, 8, 4],
        [3, 15, 0, 6, 10, 1, 13, 8, 9, 4, 5, 11, 12, 7, 2, 14]
    ],
    # fifth S-box
    [
        [2, 12, 4, 1, 7, 10, 11, 6, 8, 5, 3, 15, 13, 0, 14, 9],
        [14, 11, 2, 12, 4, 7, 13, 1, 5, 0, 15, 10, 3, 9, 8, 6],
        [4, 2, 1, 11, 10, 13, 7, 8, 15, 9, 12, 5, 6, 3, 0, 14],
        [11, 8, 12, 7, 1, 14, 2, 13, 6, 15, 0, 9, 10, 4, 5, 3]
    ],
    # sixth S-box
    [
        [12, 1, 10, 15, 9, 2, 6, 8, 0, 13, 3, 4, 14, 7, 5, 11],
        [10, 15, 4, 2, 7, 12, 9, 5, 6, 1, 13, 14, 0, 11, 3, 8],
        [9, 14, 15, 5, 2, 8, 12, 3, 7, 0, 4, 10, 1, 13, 11, 6],
        [4, 3, 2, 12, 9, 5, 15, 10, 11, 14, 1, 7, 6, 0, 8, 13]
    ],
    # seventh S-box
    [
        [4, 11, 2, 14, 15, 0, 8, 13, 3, 12, 9, 7, 5, 10, 6, 1],
        [13, 0, 11, 7, 4, 9, 1, 10, 14, 3, 5, 12, 2, 15, 8, 6],
        [1, 4, 11, 13, 12, 3, 7, 14, 10, 15, 6, 8, 0, 5, 9, 2],
        [6, 11, 13, 8, 1, 4, 10, 7, 9, 5, 0, 15, 14, 2, 3, 12]
    ],
    # eighth S-box 8
    [
        [13, 2, 8, 4, 6, 15, 11, 1, 10, 9, 3, 14, 5, 0, 12, 7],
        [1, 15, 13, 8, 10, 3, 7, 4, 12, 5, 6, 11, 0, 14, 9, 2],
        [7, 11, 4, 1, 9, 12, 14, 2, 0, 6, 10, 13, 15, 3, 5, 8],
        [2, 1, 14, 7, 4, 10, 8, 13, 15, 12, 9, 0, 3, 5, 6, 11]
    ]
]

In [57]:
def s_box_lookup(s_box, six_bits):
    row = int(six_bits[0] + six_bits[-1], 2)
    col = int(six_bits[1:5], 2)
    return format(s_box[row][col], '04b')

def process_s_boxes(expanded_block):
    # Divide the expanded_block into 6-bit sections and process through S-boxes
    output = ''
    for i in range(0, len(expanded_block), 6):
       
        six_bits = expanded_block[i:i + 6]
        # s_box = S_BOX[i // 6]
        s_box = S_BOX[0]
        output += s_box_lookup(s_box, six_bits)
    return output

s_box_output = process_s_boxes(xor_val)
print("S-box Output: ", s_box_output)

S-box Output:  11111011000110101110010100100010


S-box Output:  11111011000110101110010100100010

## Step 8: Permutation through P-box and XOR with the left half
in this section we should permute and xor the result of s_box process conducted on expanded_block 


In [58]:
P_BOX_TABLE = [16, 7, 20, 21, 29, 12, 28, 17, 1, 15, 23, 26, 5, 18, 31, 10, 2, 8, 24, 14, 32, 27, 3, 9, 19, 13, 30, 6, 22, 11, 4, 25]

#after s-box process, we should permute the result and xor it with the left side of our input
def permute_and_xor(left, right, table):
    permuted = ''.join([right[i - 1] for i in table])
    xored = xor_function(permuted, left)  
    return xored

final_left = permute_and_xor(left_half, s_box_output, P_BOX_TABLE)
print("Final Left after P-box and XOR: ", final_left)

Final Left after P-box and XOR:  10001001110011100010101000110101


Final Left after P-box and XOR:  10001001110011100010101000110101

## Step 9: Swapping the halves for the next round


In [59]:
#last part of each 16-round process conduction in DES is swapping the left and right halves of data blocks
def swap_halves(left, right):
    new_right = left
    new_left = right
    return new_left,new_right

new_left, new_right = swap_halves(final_left, right_half)
print("New Left Half: ", new_left)
print("New Right Half: ", new_right)

New Left Half:  11110000101010101111000010101010
New Right Half:  10001001110011100010101000110101


New Left Half:  11110000101010101111000010101010

New Right Half:  10001001110011100010101000110101

## Step 10: Rounds of Encryption
here we will conduct the process of encryption in DES algorithm by assembling previous steps and returning the final block


In [64]:
def rounds_of_encryption(left, right, rounded_key, num_rounds=16):
    for round_number in range(num_rounds):
        #here we will assemble our 16-round steps aim at conduction of the encryption process in each 16 iteration
        #you should to call each completed step and conduct the 16 round encryption
        expanded_right = expand_right_half(right, EXPANSION_TABLE)

        xor_output = xor_function(expanded_right, rounded_key)

        s_box_output = process_s_boxes(xor_output)

        p_box_output = permute_and_xor(left, s_box_output, P_BOX_TABLE)

        left, right = right, p_box_output

    return left + right

# Initialize with the first permutation outputs
final_block = rounds_of_encryption(left_half, right_half,subkey)
print("Final Block: ", final_block)

Final Block:  1110110110111011001100110010101111101000000101000101001011000000


Final Block:  1110110110111011001100110010101111101000000101000101001011000000

## Step 11:  Final Permutation Table

In [70]:
#base on the given diagram of DES algorithm, we have a final permutation after encryption process
FP_TABLE = [40, 8, 48, 16, 56, 24, 64, 32,
            39, 7, 47, 15, 55, 23, 63, 31,
            38, 6, 46, 14, 54, 22, 62, 30,
            37, 5, 45, 13, 53, 21, 61, 29,
            36, 4, 44, 12, 52, 20, 60, 28,
            35, 3, 43, 11, 51, 19, 59, 27,
            34, 2, 42, 10, 50, 18, 58, 26,
            33, 1, 41, 9, 49, 17, 57, 25]

def final_permutation(block, table):
    if len(block) != 64:
        raise ValueError(f"Block size must be 64 bits, but got {len(block)} bits")
    
    permuted_block = ''.join([block[i - 1] for i in table])
    return permuted_block


encrypted_output = final_permutation(final_block, FP_TABLE)
print("Encrypted Output(DES result): ", encrypted_output)
print("Inputted block:               ", input_block)


Encrypted Output(DES result):  0101010100011101011000001101000100111100110101011100101011010010
Inputted block:                0000000100100011010001010110011110001001101010111100110111101111


Encrypted Output(DES result):  
0101010100011101011000001101000100111100110101011100101011010010

Inputted block:                
0000000100100011010001010110011110001001101010111100110111101111